In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile, os
zip_path = "/content/drive/MyDrive/TrainCV.v1i.yolov11.zip"

In [ ]:
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
!cat /content/dataset/data.yaml

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 12
names: ['Kolesa', 'fire', 'insiderail', 'lep', 'outsiderail', 'people', 'perevod', 'rail', 'something', 'stolb', 'train', 'znak']

roboflow:
  workspace: cv-iexx5
  project: traincv-dnkbk
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/cv-iexx5/traincv-dnkbk/dataset/1

In [ ]:
models = ["n","m"]

model = YOLO(f"yolo11{_type}-seg.pt")

results = model.train(
  data="/content/dataset/data.yaml",
  epochs=100,
  imgsz=640,
  batch=8,
  project=f"yolo_{_type}",
  name="train",
  exist_ok=True
  )

results = model.predict(
    source="/content/dataset/test/images",
    show=False,
    save=True
)

Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=

In [ ]:
model.val()

In [ ]:
!ls /content/yolo11_training/segmentation_model/predict

ls: cannot access '/content/yolo11_training/segmentation_model/predict': No such file or directory


In [ ]:
import gradio as gr
from PIL import Image

In [ ]:
model = YOLO("/content/yolo11_training/segmentation_model/weights/best.pt")

In [ ]:
def segment_image(image):
    """
    Принимает изображение (PIL.Image), запускает YOLO,
    возвращает изображение с разметкой (также PIL.Image).
    """
    results = model.predict(source=image, save=False, show=False)
    # Берём результат и визуализируем
    result_img = results[0].plot()  # OpenCV image (numpy)
    return Image.fromarray(result_img[..., ::-1])

In [ ]:
demo = gr.Interface(
    fn=segment_image,
    inputs=gr.Image(type="pil", label="Загрузите изображение"),
    outputs=gr.Image(label="Результат сегментации"),
    title="🎯 YOLOv11 Segmentation Demo",
    description="Загрузите изображение, чтобы увидеть предсказания модели YOLOv11"
)

In [ ]:
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d549395b1dfb275537.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import matplotlib.pyplot as plt